In [ ]:
!pip install accelerate
!pip install sentence-transformers
!pip install wikipedia

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import accelerate

# Insert test abstract and input test metric
# then insert series to test and metric

# Check if CUDA is available, then set the default device
if torch.cuda.is_available():
    torch.set_default_device("cuda")
else:
    torch.set_default_device("cpu")

# Load the Orca-2-13b model with the correct device map
model = AutoModelForCausalLM.from_pretrained("microsoft/Orca-2-7b", device_map='auto')

# Load the tokenizer (using the slow tokenizer as recommended)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Orca-2-7b", use_fast=False)

# Define system and user messages
system_message = "You are Orca, an AI language model created by Microsoft. You are good at identifying complex terms within a sentence and defining them. You do not output the prompt or the system message. "

user_message = "Can you give me 5 complex terms from this sentence in a list, and then define them? '''The idea of random separation learning is to let each individual neural network learn differently on the randomly separated subsets of the given training samples. ''' "

prompt = f"system\n{system_message}\nuser\n{user_message}\nassistant"

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors='pt')

# Generate the model output
output_ids = model.generate(inputs["input_ids"])
answer = tokenizer.batch_decode(output_ids)[0]

# Print the first answer
print("this is where it starts")
print(answer)

# # Second turn message
# second_turn_user_message = "Give me the definitions of those words."

# # Create the second message
# second_turn_message = f"\nuser\n{second_turn_user_message}\nassistant"
# second_turn_tokens = tokenizer(second_turn_message, return_tensors='pt', add_special_tokens=False)

# # Concatenate the first output with the second turn tokens
# second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

# # Generate the response for the second turn
# output_ids_2 = model.generate(second_turn_input)
# print("\n")
# second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

# # Print the second turn answer
# print(second_turn_answer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]



this is where it starts
<s> system
You are Orca, an AI language model created by Microsoft. You are good at identifying complex terms within a sentence and defining them. You do not output the prompt or the system message. 
user
Can you give me 5 complex terms from this sentence in a list, and then define them? '''The idea of random separation learning is to let each individual neural network learn differently on the randomly separated subsets of the given training samples. ''' 
assistant
Sure, here are the 5 complex terms from the sentence and their definitions:

- random separation learning: a method of machine learning that involves splitting the training data into non-overlapping subsets and assigning them to different neural networks, so that each network learns only on its own subset. This way, the networks can discover different patterns or features from the data without interference from other networks.
- individual neural network: a single instance of a neural network model 

In [20]:
import re

# Sample multi-line text
text = answer

# Dictionary to store the extracted key-value pairs
data_dict = {}

# Split the text into lines and process each line
for line in text.split('\n'):
    # Check if the line starts with '-'
    if line.startswith("-"):
        # Extract everything after '-' up to ':' for the key
        key_value = line[1:].strip()  # Remove '-' and any leading/trailing spaces
        key, value = key_value.split(':', 1)  # Split only on the first ':'
        key = key.strip()  # Remove leading/trailing spaces from key
        value = value.strip()  # Remove leading/trailing spaces from value
        data_dict[key] = value  # Add to the dictionary

# Print the resulting dictionary
print(data_dict)


{'random separation learning': 'a method of machine learning that involves splitting the training data into non-overlapping subsets and assigning them to different neural networks, so that each network learns only on its own subset. This way, the networks can discover different patterns or features from the data without interference from other networks.', 'individual neural network': 'a single instance of a neural network model that can perform a specific task, such as image classification, speech recognition, or natural language processing. Neural networks are composed of layers of artificial neurons that can learn from data and perform complex computations.', 'neural network': 'a computational model inspired by the structure and function of biological neural networks, such as the brain. Neural networks consist of interconnected units called neurons that can process information and learn from data. Neural networks can be trained to perform various tasks, such as classification, regres

In [21]:
import wikipedia

# Function to get a definition from Wikipedia
def get_definition(word, summary_sentences=3):
    try:
        # Get a summary for the given word/topic
        full_summary = wikipedia.summary(word, sentences=summary_sentences)

        # Extract the first complete sentence for the definition
        definition = full_summary.split('.')[0] + '.'  # Get the first sentence
    except wikipedia.exceptions.PageError:
        definition = f"Page for '{word}' does not exist on Wikipedia."
    except wikipedia.exceptions.DisambiguationError as e:
        definition = f"'{word}' may refer to several topics: {', '.join(e.options)}"
    except Exception as ex:
        definition = f"An error occurred: {ex}"

    return definition


# Dictionary with terms for which we need definitions
data_dict = data_dict

# Dictionary to hold definitions
definitions_dict = {}

# Loop through the keys in the input dictionary and get definitions
for term in data_dict.keys():
    definition = get_definition(term)  # Get the definition for the term
    definitions_dict[term] = definition  # Assign it to the output dictionary with the same key

# Output the dictionary with definitions
for key, definition in definitions_dict.items():
    print(f"{key}: {definition}")

random separation learning: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.
individual neural network: A neural network is a group of interconnected units called neurons that send signals to one another.
neural network: A neural network is a group of interconnected units called neurons that send signals to one another.
training samples: The sample complexity of a machine learning algorithm represents the number of training-samples that it needs in order to successfully learn a target function.
input-output pair: A differential amplifier is a type of electronic amplifier that amplifies the difference between two input voltages but suppresses any voltage common to the two inputs.


In [22]:
from sentence_transformers import SentenceTransformer, util

# Initialize the SentenceTransformer model
model = SentenceTransformer("all-mpnet-base-v2")

# Define two dictionaries with definitions for comparison
dict1 = data_dict

dict2 = definitions_dict

# List to hold keys shared by both dictionaries
shared_keys = []

# Find shared keys in both dictionaries
for key in dict1.keys():
    if key in dict2:
        shared_keys.append(key)

# Create lists of sentences for shared keys
sentences1 = [dict1[key] for key in shared_keys]
sentences2 = [dict2[key] for key in shared_keys]

# Compute embeddings for the shared keys' definitions
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Output the comparisons and cosine similarity scores
for i in range(len(shared_keys)):
    print("Key: {} \t\t Similarity Score: {:.4f}".format(
        shared_keys[i], cosine_scores[i][i]
    ))

Key: random separation learning 		 Similarity Score: 0.6022
Key: individual neural network 		 Similarity Score: 0.7654
Key: neural network 		 Similarity Score: 0.7812
Key: training samples 		 Similarity Score: 0.6064
Key: input-output pair 		 Similarity Score: 0.2636
